In [1]:
#installing py
!pip install pydriller==1.15

  Using cached PyDriller-1.15-py3-none-any.whl (62 kB)
  Attempting uninstall: pydriller
    Found existing installation: PyDriller 2.0
    Uninstalling PyDriller-2.0:
      Successfully uninstalled PyDriller-2.0


In [2]:
#installing NLTK for text analysis
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\AQ38570\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\AQ38570\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\AQ38570\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\AQ38570\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\AQ38570\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nlt

True

In [13]:
#imports
import pandas as pd
import json
import os
from pydriller import RepositoryMining
from nltk import word_tokenize
from datetime import datetime
import pprint
from multiprocessing import Pool
from code_review_miner import compute_statistics
from sqlalchemy import create_engine

In [4]:
#globals
DATA_PATH = 'C:/Users/AQ38570/Desktop/work/code review and refactoring/data'
NB_PRCOCESS = 10
#change to QT for QT project 
PROJECT = 'Openstack'
#change for 'https://codereview.qt-project.org' when 
URL = 'https://review.opendev.org'
#change to 'sqlite:///QT_results.db' for QT
DATABASE_URI = 'sqlite:///Openstack_results.db'
#change to 'QT_results' for QT
SQLITE_TABLE = "Openstack_results"
#Change to QT_results.csv for QT
CSV_FILENAME = 'Openstack_results.csv'
KEYWORDS = {
    'refactoring' : ['refactor']
}
STATUS = ['abandoned-changes','merged-changes']
all_results = []

In [5]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


with Pool(NB_PRCOCESS) as p:
    for status in STATUS : 
        current_data_path = os.path.join(DATA_PATH,PROJECT,'raw data','review data',status)
        files_lists = chunks(os.listdir(current_data_path),NB_PRCOCESS)
        arguments = [(PROJECT,URL,[status],KEYWORDS,{status : files}) for files in files_lists]
        all_results += p.starmap(compute_statistics,arguments)
        

In [6]:
subject_refactor_count = 0 
desc_refactor_count = 0
global_count = 0 
rows = []
for result in all_results : 
    subject_refactor_count += result[1]['refactoring']['refactor']
    desc_refactor_count += result[2]['refactoring']['refactor']
    global_count +=result[3]
    rows += result[0]

In [15]:
df = pd.DataFrame(rows)

In [16]:
#writing results to .csv
df.to_csv(CSV_FILENAME,index=False)

In [17]:
#putting results in a sqlite table
engine = create_engine(DATABASE_URI, echo=True)
sqlite_connection = engine.connect()
df.to_sql(SQLITE_TABLE, sqlite_connection, if_exists= 'replace')
sqlite_connection.close()

2021-11-30 13:04:27,932 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-11-30 13:04:27,932 INFO sqlalchemy.engine.base.Engine ()
2021-11-30 13:04:27,933 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-11-30 13:04:27,933 INFO sqlalchemy.engine.base.Engine ()
2021-11-30 13:04:28,465 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Openstack_results")
2021-11-30 13:04:28,466 INFO sqlalchemy.engine.base.Engine ()
2021-11-30 13:04:28,467 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Openstack_results")
2021-11-30 13:04:28,467 INFO sqlalchemy.engine.base.Engine ()
2021-11-30 13:04:28,469 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Openstack_results" (
	"index" BIGINT, 
	id TEXT, 
	url TEXT, 
	branch TEXT, 
	"#reviewers" BIGINT, 
	"#messages" BIGINT, 
	"#inline_comments" BIGINT, 
	"#unsolved_inline_comments" BIGINT, 
	len_messages BIGINT, 
	churn BIGINT, 
	"#rev